### Required downloads:

In [ ]:
%pip install pandas
%pip install together

### Import sample recipes:

In [ ]:
import pandas as pd
import numpy as np


# currently a list of all ingredients that appears in our recipe list
# in future development, this should be directly retrieved from supermarkets' available product list
ingredient_df = pd.read_csv('ChatbotUI-master/supermarket_product_list.csv', index_col=0)
ingredient_list = ingredient_df.index.to_list()
discounted_ingredients = set(ingredient_df[ingredient_df['discount'].notnull()].index.to_list())
# Naive implementation: only take the number of ingredients that are currently discounted
recipe_df = pd.read_csv('ChatbotUI-master/data_recipe_unnumbered.csv')
recipe_df['discount_count'] = recipe_df['ingredient_en'].apply(lambda x: len(set(x.split(', ')).intersection(set(discounted_ingredients))))
print("recipe_df content: \n", recipe_df)

# AI Conversation setup

In [48]:
import os
from together import Together
# toaster's together key
os.environ["TOGETHER_API_KEY"] = "f00e4f3943a505ce68ff471fb2976c33040fcbb6c16874121ad76dbc53b70952" 
import json
import re

# ingredient_list = [
#     'Beef', 'Green onion', 'Garlic', 'Ginger', 'Oyster sauce', 'Sugar', 'Cornstarch', 'Oil', 'Salt', 'Soy sauce', 
#     'Black pepper powder', 'Eggs', 'Dried whitebait', 'Chicken wings', 'Potatoes', 'Onion', 'Dark soy sauce', 
#     'Light soy sauce', 'Pepper', 'Sesame oil', 'Spring chicken', 'Carrot', 'Broccoli', 'Monk fruit sugar', 
#     'Rice wine', 'Curry powder', 'Evaporated milk', 'Minced meat', 'Cold rice', 'Dried scallops', 
#     'Choy sum stems', 'Egg whites', 'Coriander', 'Seasoning', 'Rice wine', 'Chicken fillet', 'Salted fish', 
#     'Rice', 'Water spinach', 'Beef slices', 'Minced garlic', 'Chicken powder', 'Chili', 'Chicken drumsticks', 
#     'Star anise', 'Cotton tofu', 'Braising sauce', 'Palm sugar or brown sugar'
# ]

In [7]:
# stream = client.chat.completions.create(
#   model="meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
#   messages=[{"role": "user", "content": "What are some fun things to do in New York?"}],
#   stream=True,
# )

# for chunk in stream:
#   print(chunk.choices[0].delta.content or "", end="", flush=True)

In [50]:
together = Together(api_key=os.environ.get("TOGETHER_API_KEY"))

with open("ChatbotUI-master/functions.json", "r") as json_file:
    funcs = json.load(json_file)

funcs[0]["function"]["parameters"]["properties"]["include_ingredient"]["items"]["enum"] = ingredient_list
# funcs[0]["function"]["parameters"]["properties"]["exclude_ingredient"]["items"]["enum"] = ingredient_list

### Tool parameter method: 

In [ ]:
user_question = "Can you recommend me what to cook? If possible, please include ginger and potatoes. Exclude apples"

messages = [
    {"role": "system", 
     "content": 
     "You are a helpful assistant that can access external functions. "
     "The responses from these function calls will be appended to this dialogue. "
     "Please provide responses based on the information from these function calls."},
    {"role": "user", 
     "content": user_question}
]
    
# Notable issue: When user asks to exclude an item not in our built in list of ingredients, AI panics and excludes everything that isn't included. More testing required.

response = together.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=messages,
    tools=funcs,
    tool_choice="auto",
)

print(json.dumps(response.choices[0].message.model_dump()['tool_calls'], indent=2))

### Key function: recommend_recipes()

In [55]:
# main feature
def recommend_recipes(include_ingredient: list[str], 
                      exclude_ingredient: list[str], 
                      recipe_count: int=1,
                      exclude_id: list[int]=[]) -> str:
    global recipe_df
    print("Include: ", include_ingredient)
    print("Exclude: ", exclude_ingredient)
    print("Recipe_count: ", recipe_count)
    remaining_recipes = recipe_df
    # first we remove recipes containing excluded ingredients
    for ingredient in exclude_ingredient:
        # print(~remaining_recipes['ingredient_en'].str.contains(', '+ingredient+',', case=False, na=False))
        remaining_recipes = remaining_recipes[~remaining_recipes['ingredient_en'].str.contains(ingredient, case=False, na=False)]
        print("remaining_recipes: \n", 
              remaining_recipes[~remaining_recipes['ingredient_en'].str.contains(', '+ingredient+',', case=False, na=False)])
    include_set = set([i.lower() for i in include_ingredient])
    ans = []
    # Naive implementation: repeatedly select the recipe that ticks off the most items in the include list
    for i in range(max(recipe_count, 1)):
        print(f'Dish {i}:') 
        remaining_recipes['matches'] = remaining_recipes['ingredient_en'].apply(lambda x: len(set(x.split(', ')).intersection(set(include_set))))
        remaining_recipes = remaining_recipes.sort_values(by=["matches", "discount_count"], ascending=False)
        print("remaining_recipes: \n", remaining_recipes)
        head = remaining_recipes.head(1)
        print("Head: ", head)
        if head.empty:  # no recipe found
            break
        remaining_recipes = remaining_recipes.drop(head.index[0])
        selected_recipe = head.T[head.index[0]].to_list()
        selected_recipe.append(head.index[0])
        ans.append(selected_recipe)
        print(selected_recipe)
        include_set = include_set - set([ing.lower() for ing in ans[-1][1].split(', ')])
    return_recipes = ""
    for i in range(len(ans)):
        return_recipes += f"Recipe {i+1}: {ans[i][0]} (recipe_id {ans[i][3]})\nIngredients: {ans[i][1]}\nInstructions: \n{ans[i][2]}\n\n"
    return return_recipes

In [45]:
messages

[{'role': 'system',
  'content': 'You are a helpful assistant that can access external functions. The responses from these function calls will be appended to this dialogue. Please provide responses based on the information from these function calls.'},
 {'role': 'user',
  'content': 'Can you recommend me what to cook? If possible, please include ginger and potatoes. Exclude apples'},
 {'tool_call_id': 'call_dfg60nl321dgki62rhyl2a98',
  'role': 'tool',
  'name': 'recommend_recipes',
  'content': [['fried beef with spring onion',
    'Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder',
    ' 1. Marinate the beef. 2. Chop the ginger and garlic.\n\nWash and cut the green onions.\nSauté the ginger and green onions until fragrant.\nQuickly stir-fry the beef.\nOnce it starts to change color, add the green onions and stir-fry.\nAdd the prepared sauce and cook until done.',
    '蔥炒牛肉',
    '牛肉, 大蔥, 蒜頭, 薑 , 蠔油, 糖, 生粉, 油, 鹽, 生抽, 黑胡椒粉',
 

In [56]:
tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        if function_name == "recommend_recipes":
            function_response = recommend_recipes(
                include_ingredient=function_args.get("include_ingredient"),
                exclude_ingredient=function_args.get("exclude_ingredient"),
                recipe_count=function_args.get("recipe_count"),
                exclude_id=function_args.get("exclude_id"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
    response = together.chat.completions.create(
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",
        messages=messages,
    )
    print(json.dumps(response.choices[0].message.model_dump(), indent=2))


Include:  ['Ginger', 'Potatoes']
Exclude:  ['Apples']
Recipe_count:  3
remaining_recipes: 
                                        title_en  \
0                  fried beef with spring onion   
1    Whitebait and Green Onion Omelet Triangles   
2           Braised Chicken Wings with Potatoes   
3          Simple Hong Kong Style Curry Chicken   
4          Hong Kong Style Steamed Pork and Egg   
5  Hong Kong Style Scallop Egg White Fried Rice   
6            Salted Fish and Chicken Fried Rice   
7     Stir-Fried Water Spinach with Beef Slices   
8           Homemade Braised Chicken Drumsticks   
9                   Stir-Fried Beef with Onions   

                                       ingredient_en  \
0  Beef, green onion, garlic, ginger, oyster sauc...   
1  Eggs, green onions, dried whitebait, salt, bla...   
2  Chicken wings, potatoes, onion, dark soy sauce...   
3  Spring chicken, onion, carrot, potato, broccol...   
4  Eggs, water, minced meat, soy sauce, salt, gre...   
5   Cold r

In [43]:
print(function_response)

Recipe 1: Braised Chicken Wings with Potatoes
Ingredients: Chicken wings, potatoes, onion, dark soy sauce, light soy sauce, pepper, sugar, sesame oil
Instructions: 1 Thaw the chicken wings, rinse, and drain. Mix with marinade and let sit for about half an hour.

2 Slice the onion.

3 Cut the potatoes into chunks.

4 Heat a pan over medium heat, add oil, and fry the chicken skin side down. Flip and fry until both sides are colored, then set aside.

5 Heat a pan over medium heat, add oil, and sauté ginger slices and onion. Add potatoes and fry until fragrant.

6 Add the chicken wings, remaining marinade, and a bowl of water.

7 Once the sauce boils, reduce to low heat, cover, and simmer for about 20 minutes.

8 Stir and taste. Adjust seasoning with salt if needed.

Recipe 2: fried beef with spring onion
Ingredients: Beef, green onion, garlic, ginger, oyster sauce, sugar, cornstarch, oil, salt, soy sauce, black pepper powder
Instructions:  1. Marinate the beef. 2. Chop the ginger and garl

In [42]:
ingredient_df = pd.read_csv('supermarket_product_list.csv', index_col=0)
print("ingredient_df content: \n", ingredient_df)
print("ingredient_df foods: \n", ingredient_df.index.to_list())
discounted_ingredients = set(ingredient_df[ingredient_df['discount'].notnull()].index.to_list())
print("discounted_ingredients: \n", discounted_ingredients)

ingredient_df content: 
                           expiry_date discount retail_location
food_name                                                     
Beef                       19.10.2024      NaN       PARKnSHOP
Green onion                20.10.2024      NaN            AEON
Garlic                     19.10.2024      NaN            AEON
Ginger                     20.10.2024      20%            AEON
Oyster sauce               20.10.2024      30%       PARKnSHOP
Sugar                      19.10.2024      30%        Wellcome
Cornstarch                 19.10.2024      NaN       PARKnSHOP
Oil                        20.10.2024      30%       PARKnSHOP
Salt                       19.10.2024      30%       PARKnSHOP
Soy sauce                  19.10.2024      30%            AEON
Black pepper powder        19.10.2024      10%       PARKnSHOP
Eggs                       19.10.2024      NaN       PARKnSHOP
Dried whitebait            20.10.2024      NaN        Wellcome
Chicken wings              19.